In [ ]:
import tensorflow as tf
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2

# Step 1: Prepare your dataset directory structure
data_directory = "../EuroSAT_RGB/"
image_size = (128, 128)  # Adjust image size as needed

# Step 2: Load and preprocess data
def load_and_preprocess_data(data_dir, image_size):
    images = []
    labels = []

    for class_name in os.listdir(data_dir):
        class_dir = os.path.join(data_dir, class_name)
        if os.path.isdir(class_dir):
            for filename in os.listdir(class_dir):
                if filename.lower().endswith((".tif", ".jpg", ".png")):
                    img = cv2.imread(os.path.join(class_dir, filename), cv2.IMREAD_COLOR)

                    if img is not None:
                        img = cv2.resize(img, image_size)
                        img = img / 255.0  # Normalize pixel values to [0, 1]
                        images.append(img)
                        labels.append(class_name)

    return np.array(images), np.array(labels)

images, labels = load_and_preprocess_data(data_directory, image_size)

# Randomize the data
random_indices = np.random.permutation(len(images))
images = images[random_indices]
labels = labels[random_indices]

# Step 3: Label encoding
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

# Take 1/10th of the data for training and validation
subset_size = len(images) // 10
x_train, x_val, y_train, y_val = train_test_split(images[:subset_size], encoded_labels[:subset_size], test_size=0.2, random_state=42)

# Step 4: Build a simple CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_size + (3,)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Step 5: Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train the model
batch_size = 32
epochs = 10

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))

# Step 7: Evaluate the model
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print("Validation accuracy:", test_accuracy)

# Step 8: Save the model
model.save("field_or_not.h5")